<a href="https://colab.research.google.com/github/lsgotti/TCC_Python/blob/main/undersampling_crosval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#carrega imports e instala o scikit-multiflow
!pip install -U scikit-multiflow
# Imports
from skmultiflow.data import FileStream
from skmultiflow.data import SEAGenerator
from skmultiflow.evaluation import EvaluatePrequential
from skmultiflow.bayes import NaiveBayes
from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.trees import HoeffdingAdaptiveTreeClassifier
from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
from skmultiflow.meta import LearnPPNSEClassifier
from skmultiflow.drift_detection import ADWIN
from skmultiflow.meta import OzaBaggingClassifier

from sklearn.linear_model import SGDClassifier

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn import svm
import pandas as pd
from sklearn import datasets, linear_model
from matplotlib import pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from skmultiflow.lazy import KNNClassifier

from google.colab import files

import numpy as np
from skmultiflow.data.file_stream import FileStream
from skmultiflow.data.data_stream import DataStream
from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.evaluation import EvaluatePrequential
import pandas as pd
import io
from google.colab import files
from skmultiflow.trees import HoeffdingTree
from skmultiflow.meta import AdaptiveRandomForestClassifier
from datetime import datetime, date

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#segunda parte
#importa arquivo para teste
uploaded = files.upload()
#Importa o arquivo para o colab
import io

#abre o arquivo no dataframe
df = pd.read_csv(io.BytesIO(uploaded['tomcat3.csv']))

Saving tomcat3.csv to tomcat3.csv


In [ ]:
np.set_printoptions(formatter={'float_kind':'{:25f}'.format})

#Etapa de downsampling

#Divide o dataframe em codigos 0 e 1, apenas o 1 sera usado

m=df.code==0
df_code0=df[m]
df_code1=df[~m]

print(df_code1.contains_bug.value_counts())

# Separa classes minoritarias e majoritarias
df_majority = df_code1[df_code1.contains_bug==0]
df_minority = df_code1[df_code1.contains_bug==1]


# Downsample de classe majoritaria
df_majority_downsampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=123)

# Combina classe minoritaria com downsample da majoritaria
df_downsampled_partial = pd.concat([df_majority_downsampled, df_minority])

print(df_downsampled_partial.contains_bug.value_counts())
df_downsampled = pd.concat([df_downsampled_partial, df_code0])

# novo count de classes
print(df_downsampled.contains_bug.value_counts())

#embaralha o dataframe
df_downsampled = df_downsampled.sample(frac=1)

df_traindata = df_downsampled[['contains_bug']]
df_downsampled.drop('contains_bug', inplace=True, axis=1)
df_downsampled.drop('code', inplace=True, axis=1)
df_downsampled.drop('days_to_first_fix', inplace=True, axis=1)
df_downsampled.drop('repository_name', inplace=True, axis=1)

df_traindata2 = df_traindata.to_numpy().reshape(df_downsampled.shape[0],)
df_downsampled2 = df_downsampled.to_numpy()
df_downsampled2.shape, df_traindata2.shape

0    16727
1     5207
Name: contains_bug, dtype: int64
0    5207
1    5207
Name: contains_bug, dtype: int64
0    18907
1    10414
Name: contains_bug, dtype: int64


((29321, 16), (29321,))

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import math
import numpy as np
import copy as cp
import matplotlib.pyplot as plt

import seaborn as sns
from typing import Tuple
from sklearn.metrics import confusion_matrix

kfold = KFold(n_splits=10, random_state=42, shuffle=True)

model_ = DecisionTreeClassifier()

actual_classes = np.empty([0], dtype=int)
predicted_classes = np.empty([0], dtype=int)

rec1=  0
rec0 = 0
gmean = 0
recall0list = []
recall1list = []
gmeanList = []

for train_ndx, test_ndx in kfold.split(df_downsampled2):
  train_X, train_y, test_X, test_y = df_downsampled2[train_ndx], df_traindata2[train_ndx], df_downsampled2[test_ndx], df_traindata2[test_ndx]
  actual_classes = np.empty([0], dtype=int)
  predicted_classes = np.empty([0], dtype=int)

  actual_classes = np.append(actual_classes, test_y)

  model_.fit(train_X, train_y)
  predicted_classes = np.append(predicted_classes, model_.predict(test_X))

  conf_mat = confusion_matrix(actual_classes,predicted_classes)

  print(conf_mat)
  currentrec1 = conf_mat[0][0]/(conf_mat[0][0]+conf_mat[1][0])
  currentrec0 = conf_mat[1][1]/(conf_mat[1][1]+conf_mat[0][1])
  currentgmean = math.sqrt((conf_mat[0][0]/(conf_mat[0][0]+conf_mat[1][0])) * (conf_mat[1][1]/(conf_mat[1][1]+conf_mat[0][1])))
  recall0list.append(currentrec0)
  recall1list.append(currentrec1)
  gmeanList.append(currentgmean)
  rec1 += conf_mat[0][0]/(conf_mat[0][0]+conf_mat[1][0])
  rec0 += conf_mat[1][1]/(conf_mat[1][1]+conf_mat[0][1])
  gmean += math.sqrt((conf_mat[0][0]/(conf_mat[0][0]+conf_mat[1][0])) * (conf_mat[1][1]/(conf_mat[1][1]+conf_mat[0][1])))

  print(currentrec0, currentrec1, currentgmean)

rec1 = rec1/10
rec0 = rec0/10
gmean = gmean/10
recall0list.append(rec0)
recall1list.append(rec1)
gmeanList.append(gmean)

print("rec0, rec1 e gmean")
print(rec0, rec1, gmean)

results = pd.DataFrame(
    {'recall0': recall0list,
     'recall1': recall1list,
     'gmean': gmeanList
    })

results.to_csv('results.csv')
files.download('results.csv')

[[1586  321]
 [ 226  800]]
0.7136485280999108 0.8752759381898455 0.7903413091636117
[[1565  316]
 [ 235  816]]
0.7208480565371025 1.74472038263429 0.791667441571749
[[1594  297]
 [ 248  793]]
0.7275229357798165 2.6100841177048655 0.793455710834532
[[1648  273]
 [ 254  757]]
0.7349514563106796 3.476540479429366 0.7979996020544589
[[1562  316]
 [ 264  790]]
0.7142857142857143 4.331962166176353 0.7816747984510969
[[1603  307]
 [ 227  795]]
0.721415607985481 5.207918450329359 0.7949393280628045
[[1570  332]
 [ 250  780]]
0.7014388489208633 6.070555812966722 0.777873613567449
[[1562  290]
 [ 242  838]]
0.7429078014184397 6.936409471503308 0.8020283273136474
[[1546  304]
 [ 233  849]]
0.7363399826539462 7.805437015067108 0.7999373265160027
[[1606  309]
 [ 224  793]]
0.7196007259528131 8.68303264348241 0.7946813520529253
rec0, rec1 e gmean
0.7232959657944766 0.8683032643482409 0.7924598809588277


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(conf_mat)
import math
rec1=  0
rec0 = 0
gmean = 0
rec1 += conf_mat[0][0]/(conf_mat[0][0]+conf_mat[1][0])
rec0 += conf_mat[1][1]/(conf_mat[1][1]+conf_mat[0][1])
gmean += math.sqrt(rec0 * rec1)

print(rec0, rec1, gmean)

[[15470  1882]
 [ 1190  6880]]
0.7852088564254737 0.9285714285714286 0.8538867076714216
